In [7]:
#Importing all the required libraries
import selenium
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

###### Assignment 3.1:
1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user.
For e.g. If user input is ‘guitar’. Then search for guitars.

2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and have it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [102]:
#Activating the Chrome browser
driver=webdriver.Chrome('C:/Users/ak38/Desktop/FlipDataStore/chromedriver')

#Opening the home page of Amazon.in
driver.get("https://www.amazon.in/")

In [103]:
#User input 
user_in=input("Enter the product to be searched:")

Enter the product to be searched:Hard disk


In [107]:
search_bar=driver.find_element_by_id("twotabsearchtextbox")  #Locating the search bar
search_bar.clear()                #Clear the search bar
search_bar.send_keys(user_in)   #Product search via user input

search_button=driver.find_element_by_id("nav-search-submit-button").click()  #Submit button click

In [105]:
#Fetching the URLs of each product
urls=[]

for u in driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]'):
    urls.append(u.get_attribute("href"))   #Fetching the URLs from the website 


len(urls)

21

In [106]:
data={}  #Initially storing in the form of dictionary, (key,values) pair
#Creating the DataFrame to store the data 
df=pd.DataFrame(columns=['Product_Name', 'Brand', 'Rating', 'No.of_Ratings', 'Price', 'Return/Exchange', 'Expected_Delivery', 'Product URL', 'Availability'])

for url in urls[0:10]:  #Iterating through the product URLs fetched above to get each individual product details
    driver.get(url)   #Fetching the product urls from the list above
    time.sleep(3)     #sleeping for 3 seconds, letting the website load if the internet connection is slower.
    
    #Fetching the Title/Name of the product with simple description
    try:
        data["Product_Name"]=driver.find_element_by_id('title_feature_div').text
    except NoSuchElementException:
        data["Product_Name"]=driver.find_element_by_id('productTitle').text
    
    #Fetching the Brand Name of the product, if not found- brand name is taken from the title of the product
    try:
        data["Brand"]=driver.find_element_by_xpath('//div[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span').text
    except NoSuchElementException:
        data["Brand"]=data["Product_Name"].split("-")[0]
    
    #Fetching the Rating of the product    
    try:
        data['Rating']=driver.find_element_by_xpath('//table[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/td').text
    except NoSuchElementException:
        data['Rating']='-'
    
    #Fetching the Number of Ratings present for the product
    try:
        data['No.of_Ratings']=driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span').text
    except NoSuchElementException:
        data['No.of_Ratings']="-"
        
    #Fetching the Price of each product after the discounts are applied
    try:
        data['Price']=driver.find_element_by_id('priceblock_ourprice').text
    except NoSuchElementException:
        data['Price']=driver.find_element_by_id('priceblock_dealprice').text
    except NoSuchElementException:
        data["Price"]="-"
    
    #Fetching the Return/Exchange Offers for that perticular product   
    try:
        data['Return/Exchange']=driver.find_element_by_id('sopp-buyback-label').text
    except NoSuchElementException:
        data['Return/Exchange']='-'
        
    #Fecthing the Expected Delivery date if the product is purchased today    
    try:
        data['Expected_Delivery']=driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text
    except NoSuchElementException:
        data['Expected_Delivery']="-"
        
    data["Product URL"]=url
    
    #Fetching the Availibity data for each of the product
    try:
        data["Availability"]=driver.find_element_by_xpath('//div[@id="availability"]/span').text
    except NoSuchElementException:
        data["Availability"]="-"
            
    
    
    df=df.append(data, ignore_index=True)      #appending the data to dataframe

,Product_Name,Brand,Rating,No.of_Ratings,Price,Return/Exchange,Expected_Delivery,Product URL,Availability
0,Seagate Expansion 1.5 TB External HDD - USB 3....,Seagate,4.4 out of 5 stars,"98,426 ratings","₹ 4,299.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Expansion-Portab...,In stock.
1,Western Digital Elements 1.5 TB Portable Exter...,Western Digital,4.4 out of 5 stars,"13,027 ratings","₹ 4,099.00",-,"Saturday, June 12",https://www.amazon.in/Elements-Portable-Extern...,In stock.
2,Seagate Backup Plus Slim 2 TB External HDD – U...,Seagate,4.5 out of 5 stars,"21,361 ratings","₹ 5,499.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Backup-External-...,In stock.
3,Seagate Expansion 1.5 TB External HDD - USB 3....,Seagate,4.4 out of 5 stars,"98,426 ratings","₹ 4,299.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Expansion-Portab...,In stock.
4,Lenovo Portable 1TB External Hard Disk Drive H...,Lenovo,4.4 out of 5 stars,358 ratings,"₹ 3,699.00",-,"Saturday, June 12",https://www.amazon.in/Lenovo-Portable-External...,In stock.
5,ADATA HV320 1TB Sleek Light Portable USB 3.1 E...,A-DATA,3.8 out of 5 stars,97 ratings,"₹ 3,969.00",-,"Saturday, June 12",https://www.amazon.in/ADATA-Portable-External-...,In stock.
6,Western Digital WDBHDW0020BBK-EESN Elements 2T...,Western Digital,4.4 out of 5 stars,"13,027 ratings","₹ 5,249.00",-,"Saturday, June 12",https://www.amazon.in/Elements-Portable-Extern...,In stock.
7,Transcend StoreJet 2TB Slim External Hard Driv...,Transcend,4.5 out of 5 stars,"2,842 ratings","₹ 6,599.00",-,"Sunday, June 13",https://www.amazon.in/Transcend-StoreJet-Slim-...,Only 1 left in stock.
8,Western Digital WD 1TB My Passport Portable Ex...,Western Digital,4.4 out of 5 stars,"9,031 ratings","₹ 3,899.00",-,"Saturday, June 12",https://www.amazon.in/Passport-Portable-Extern...,In stock.
9,Seagate Portable 1TB External Hard Drive HDD –...,Seagate,4.4 out of 5 stars,"22,719 ratings","₹ 3,899.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Portable-Externa...,In stock.


In the above question, now scrape the following details of each product listed in first 3 pages of your search results and have it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [110]:
#Fetching the URLs of each product from first 3 pages
urls=[]  #Storing the fetched urls
i=0  #index for the page number reference

while i<=2:    #while page is less than or equal to 2 perform the below actions
    for u in driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]'):  
        urls.append(u.get_attribute("href"))   #getting the urls and iterating 
        
    try:
        next=driver.find_element_by_xpath('//li[@class="a-last"]/a').click()  #Going to next page by clicking on Next button on the bottom
        time.sleep(3)   #Sleep for 3 sec while the webpage loads completely
    except NoSuchElementException:  #When the page ends
        pass   #pass the exception
        time.sleep(3)
    i+=1   #increse by 1 page

        
        
len(urls)   #get the no.of ulrs fetched

68

In [111]:
urls

['https://www.amazon.in/Seagate-Expansion-Portable-External-Playstation/dp/B007IREFE0/ref=sr_1_1?dchild=1&keywords=Hard+disk&qid=1622914066&sr=8-1',
 'https://www.amazon.in/Elements-Portable-External-Drive-Black/dp/B06XDKWLJH/ref=sr_1_2?dchild=1&keywords=Hard+disk&qid=1622914066&sr=8-2',
 'https://www.amazon.in/Seagate-Backup-External-Drive-Portable/dp/B07M67W1PH/ref=sr_1_3?dchild=1&keywords=Hard+disk&qid=1622914066&sr=8-3',
 'https://www.amazon.in/Seagate-Expansion-Portable-External-Playstation/dp/B007IREFE0/ref=sxin_9?asc_contentid=amzn1.osa.d4a0fd3a-0965-4585-ac7f-3029b8b1e48f.A21TJRUUN4KGV.en_IN&asc_contenttype=article&ascsubtag=amzn1.osa.d4a0fd3a-0965-4585-ac7f-3029b8b1e48f.A21TJRUUN4KGV.en_IN&creativeASIN=B007IREFE0&cv_ct_cx=Hard+disk&cv_ct_id=amzn1.osa.d4a0fd3a-0965-4585-ac7f-3029b8b1e48f.A21TJRUUN4KGV.en_IN&cv_ct_pg=search&cv_ct_we=asin&cv_ct_wn=osp-single-source-earns-comm&dchild=1&keywords=Hard+disk&linkCode=oas&pd_rd_i=B007IREFE0&pd_rd_r=c819a6f8-448f-4ca0-9a4e-eedb1dfbd88a&

In [121]:
data={}  #Initially storing in the form of dictionary, (key,values) pair
#Creating the DataFrame to store the data 
df=pd.DataFrame(columns=['Product_Name', 'Brand', 'Rating', 'No.of_Ratings', 'Price', 'Return/Exchange', 'Expected_Delivery', 'Product URL', 'Availability'])

for url in urls:  #Iterating through the product URLs fetched above to get each individual product details
    driver.get(url)   #Fetching the product urls from the list above
    time.sleep(3)     #sleeping for 3 seconds, letting the website load if the internet connection is slower.
    
    #Fetching the Title/Name of the product with simple description
    try:
        data["Product_Name"]=driver.find_element_by_id('title_feature_div').text
    except NoSuchElementException:
        data["Product_Name"]=driver.find_element_by_id('productTitle').text
    
    #Fetching the Brand Name of the product, if not found- brand name is taken from the title of the product
    try:
        data["Brand"]=driver.find_element_by_xpath('//div[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span').text
    except NoSuchElementException:
        data["Brand"]=data["Product_Name"].split("-")[0]
    
    #Fetching the Rating of the product    
    try:
        data['Rating']=driver.find_element_by_xpath('//table[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/td').text.replace("\n", " ")
    except NoSuchElementException:
        data['Rating']='-'
    
    #Fetching the Number of Ratings present for the product
    try:
        data['No.of_Ratings']=driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span').text
    except NoSuchElementException:
        data['No.of_Ratings']="-"
        
    #Fetching the Price of each product after the discounts are applied
    try:
        data['Price']=driver.find_element_by_id('priceblock_ourprice').text
    except NoSuchElementException:
        try:
            data['Price']=driver.find_element_by_id('priceblock_dealprice').text
        except NoSuchElementException:
            try:
                data['Price']=driver.find_element_by_id('priceblock_saleprice').text
            except NoSuchElementException:
                data["Price"]="-"
    
    #Fetching the Return/Exchange Offers for that perticular product   
    try:
        data['Return/Exchange']=driver.find_element_by_id('sopp-buyback-label').text
    except NoSuchElementException:
        data['Return/Exchange']='-'
        
    #Fecthing the Expected Delivery date if the product is purchased today    
    try:
        data['Expected_Delivery']=driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text
    except NoSuchElementException:
        data['Expected_Delivery']="-"
        
    data["Product URL"]=url
    
    #Fetching the Availibity data for each of the product
    try:
        data["Availability"]=driver.find_element_by_xpath('//div[@id="availability"]/span').text
    except NoSuchElementException:
        data["Availability"]="-"
            
    
    
    df=df.append(data, ignore_index=True)      #appending the data to dataframe

In [122]:
pd.set_option('display.max_rows', None)   #this code helps to view all the rows without hiding them
df     #Product details from 3 pages of the product website

,Product_Name,Brand,Rating,No.of_Ratings,Price,Return/Exchange,Expected_Delivery,Product URL,Availability
0,Seagate Expansion 1.5 TB External HDD - USB 3....,Seagate,"67,129 ratings 4.4 out of 5 stars","67,129 ratings","₹ 4,299.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Expansion-Portab...,In stock.
1,Western Digital Elements 1.5 TB Portable Exter...,Western Digital,"13,032 ratings 4.4 out of 5 stars","13,032 ratings","₹ 4,099.00",-,June 8 - 12,https://www.amazon.in/Elements-Portable-Extern...,In stock.
2,Seagate Backup Plus Slim 2 TB External HDD – U...,Seagate,"21,424 ratings 4.5 out of 5 stars","21,424 ratings","₹ 5,499.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Backup-External-...,In stock.
3,Seagate Expansion 1.5 TB External HDD - USB 3....,Seagate,"67,129 ratings 4.4 out of 5 stars","67,129 ratings","₹ 4,299.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Expansion-Portab...,In stock.
4,Lenovo Portable 1TB External Hard Disk Drive H...,Lenovo,358 ratings 4.4 out of 5 stars,358 ratings,"₹ 3,699.00",-,"Saturday, June 12",https://www.amazon.in/Lenovo-Portable-External...,In stock.
5,ADATA HV320 1TB Sleek Light Portable USB 3.1 E...,A-DATA,97 ratings 3.8 out of 5 stars,97 ratings,"₹ 3,969.00",-,"Saturday, June 12",https://www.amazon.in/ADATA-Portable-External-...,In stock.
6,Western Digital WDBHDW0020BBK-EESN Elements 2T...,Western Digital,"13,032 ratings 4.4 out of 5 stars","13,032 ratings","₹ 5,249.00",-,June 14 - 18,https://www.amazon.in/Elements-Portable-Extern...,"In stock on June 9, 2021."
7,Transcend StoreJet 2TB Slim External Hard Driv...,Transcend,"2,842 ratings 4.5 out of 5 stars","2,842 ratings","₹ 7,299.00",-,"Saturday, June 12",https://www.amazon.in/Transcend-StoreJet-Slim-...,Only 2 left in stock.
8,Western Digital WD 1TB My Passport Portable Ex...,Western Digital,"9,031 ratings 4.4 out of 5 stars","9,031 ratings","₹ 3,899.00",-,"Saturday, June 12",https://www.amazon.in/Passport-Portable-Extern...,In stock.
9,Seagate Portable 1TB External Hard Drive HDD –...,Seagate,"22,722 ratings 4.4 out of 5 stars","22,722 ratings","₹ 3,899.00",-,"Saturday, June 12",https://www.amazon.in/Seagate-Portable-Externa...,In stock.


In [123]:
#Rating Column must be cleaned during preprocessing of the data

In [124]:
#Saving the result as CSV file, ProductDetails is the file name

df.to_csv("ProductDetails.csv")

###### Assignment 3.3:
    Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.


In [159]:
#Activating the Chrome browser
driver=webdriver.Chrome('C:/Users/ak38/Desktop/FlipDataStore/chromedriver')

#Opening the home page of Amazon.in
driver.get("https://images.google.com/")

In [161]:
#User input, It can be any images you want to download from google
user_in=input("Enter the product to be searched:")

Enter the product to be searched:Machine Learning


In [162]:
search_bar=driver.find_element_by_name('q')  #Locating the search bar
search_bar.clear()                #Clear the search bar
search_bar.send_keys(user_in)   #Product search via user input

search_button=driver.find_element_by_xpath('//span[@class="z1asCe MZy1Rb"]').click()  #Submit button click

In [155]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import sys
import os

In [163]:
driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
time.sleep(4)

In [165]:
#parsing
soup = BeautifulSoup(driver.page_source, 'html.parser')

#closing web browser
#driver.close()

#scraping image urls with the help of image tag and class used for images
img_tags = soup.find_all("img", class_="rg_i")

count = 0
for i in img_tags[:120]: #for loop iterating for 120 times as we need 100 images
    #print(i['src'])
    try:
		#passing image urls one by one and downloading, images downloaded to C:/users/ak38
        urllib.request.urlretrieve(i['src'], str(count)+".jpg")
        count+=1
        print("Number of images downloaded = "+str(count),end='\r')
    except Exception as e:
        pass

###### Assignment 3.4:
    Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. 
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.

In [389]:
#Activating the Chrome Browser
driver=webdriver.Chrome('C:/Users/ak38/Desktop/FlipDataStore/chromedriver')
driver.get("https://www.flipkart.com/")      #Opening the Home page of flipkart

In [391]:
#User input
user_in=input("Enter the product to be searched:")

Enter the product to be searched:Smart Phones


In [393]:
product_search=driver.find_element_by_name("q")        #Locating the product search bar by class name
product_search.clear()        #Clearing the search bar to make sure there is no history search
product_search.send_keys(user_in)        #Searching the product
Search_button=driver.find_element_by_class_name("L0Z3Pu")  #Locating search button
Search_button.click()     #Click

In [394]:
#Fetching the Product URLs from the 1st page of flipkart smartphone search 
urls=[]

for u in driver.find_elements_by_xpath('//a[@class="_1fQZEK"]'):
    urls.append(u.get_attribute("href"))
len(urls)

24

In [236]:
#creating a dictionary to store the key value pairs 
data={}

df=pd.DataFrame(columns=['Brand Name', 'Smartphone name', 'Colour', 'RAM', 'Storage(ROM)', 'Primary Camera', 'Secondary Camera', 
                         'Display Size', 'Display Resolution', 'Processor', 'Processor Cores', 'Battery Capacity', 'Price', 'Product URL'])


for url in urls:  #Iterating through the urls to get the required table data
    driver.get(url)
    time.sleep(3)
    
    #clicking on read more button to get all the table attributes 
    #read_more=driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')
    #read_more.click()
    #time.sleep(2)
    #read more button click is not working hence skipping it 
    
    #Fetching the Smartphone name
    try:
        data['Smartphone name']=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split("(")[0]
    except NoSuchElementException:
        data['Smartphone name']='-'
    
    #Fetching the Brand name
    data['Brand Name']=data['Smartphone name'].split(" ")[0]
    
    #Fetching the Colour
    try:
        data['Colour']=driver.find_element_by_xpath('//div[@class="_1UhVsV _3AsE0T"]/div/table/tbody/tr[4]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Colour']='-'
    
    #Fetching the RAM
    try:
        data['RAM']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        data['RAM']='-'
        
    #Fetching the ROM
    try:
        data['Storage(ROM)']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr/td[2]/ul/li').text
    except NoSuchElementException:
        data['Storage(ROM)']='-'
    
    #Fetching the Primary Camera
    try:
        data['Primary Camera']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Primary Camera']='-'
    
    #Fetching the Secondary Camera
    try:
        data['Secondary Camera']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Secondary Camera']='-'
    
    #Fetching the Display Size
    try:
        data['Display Size']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Display Size']='-'
    
    #Fetching the Display Resolution
    try:
        data['Display Resolution']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Display Resolution']='-'
    
    #Fetching the Processor
    try:
        data['Processor']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Processor']='-'
    
    #Fetching the Processor Cores
    try:
        data['Processor Cores']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]/ul/li').text
    except NoSuchElementException:
        data['Processor Cores']='-'
    
    #Fetching the Battery Capacity
    try:
        data['Battery Capacity']=driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr/td[2]/ul/li').text
    except NoSuchElementException:
        data['Battery Capacity']='-'
    
    #Fetching the Price
    try:
        data['Price']=driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]').text
    except NoSuchElementException:
        data['Price']='-'
    
    #Fetching the Product URL
    data['Product URL']=url
    

    df=df.append(data, ignore_index=True)                                   #Storing in dataframe by Appending all the data one by one
    #index+=1                                                            #Count/iteration

df

,Brand Name,Smartphone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product URL
0,Apple,Apple iPhone XR,(PRODUCT)RED,-,-,-,-,-,-,-,-,-,"₹42,999",https://www.flipkart.com/apple-iphone-xr-produ...
1,Apple,Apple iPhone SE,White,-,-,-,-,-,-,-,-,-,"₹46,999",https://www.flipkart.com/apple-iphone-se-white...
2,-,-,-,-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/realme-narzo-20-glory...
3,realme,realme C11,Rich Green,-,-,-,-,-,-,-,-,-,"₹7,499",https://www.flipkart.com/realme-c11-rich-green...
4,POCO,POCO C3,Matte Black,-,-,-,-,-,-,-,-,-,"₹7,499",https://www.flipkart.com/poco-c3-matte-black-3...
5,realme,realme C21,Cross Black,-,-,-,-,-,-,-,-,-,"₹7,999",https://www.flipkart.com/realme-c21-cross-blac...
6,realme,realme C21,Cross Blue,-,-,-,-,-,-,-,-,-,"₹8,999",https://www.flipkart.com/realme-c21-cross-blue...
7,realme,realme C21,Cross Black,-,-,-,-,-,-,-,-,-,"₹8,999",https://www.flipkart.com/realme-c21-cross-blac...
8,SAMSUNG,SAMSUNG Galaxy F41,Fusion Green,-,-,-,-,-,-,-,-,-,"₹14,499",https://www.flipkart.com/samsung-galaxy-f41-fu...
9,POCO,POCO X3,Shadow Gray,-,-,-,-,-,-,-,-,-,"₹18,999",https://www.flipkart.com/poco-x3-shadow-gray-1...


###### Assignment 3.5:
    Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [277]:
#Activating the Chrome Browser
driver=webdriver.Chrome('C:/Users/ak38/Desktop/FlipDataStore/chromedriver')
driver.get("https://www.google.co.in/maps?hl=en&tab=rl")      #Opening the Google Maps URL

In [306]:
#User input 
#Enter the city/place where the geo coordinates are required
user_in=input("Search Google Maps:")  

Search Google Maps:Chennai


In [307]:
Search=driver.find_element_by_name('q')  #fetch the search bar
Search.clear()   #Clear the search bar
Search.send_keys(user_in)   #send the user input
time.sleep(2)   #Sleep for 2 seconds to load 
Search_button=driver.find_element_by_id('searchbox-searchbutton')   #Fetch the search button

Search_button.click()  #click on the search button

In [308]:
#getting the element and url which contains the geo coordinates(latitude, longitude) 
url_with_coordinates=driver.find_element_by_xpath('//div[@id="vasquette"]/div/div/div/div/div[2]/div/a').get_attribute('href')   
url_with_coordinates

'https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https%3A%2F%2Fwww.google.co.in%2Fmaps%2Fplace%2FChennai%2C%2BTamil%2BNadu%2F%4013.0480438%2C79.9288002%2C10z%2Fdata%3D!3m1!4b1!4m5!3m4!1s0x3a5265ea4f7d3361%3A0x6e61a70b6863d433!8m2!3d13.0826802!4d80.2707184%3Fhl%3Den&service=local&ec=GAZAcQ'

In [309]:
#Extracting the [latitude, logitude] data from the above url
coordinates=url.split("%2F%40")[1].split("%2C11z")[0].replace("%2C", ",").split(",")[:2]
coordinates

['13.0480438', '79.9288002']

###### Assignment 3.6:
    Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –September 20) from trak.in.

In [320]:
#Activating the Chrome Browser
driver=webdriver.Chrome('C:/Users/ak38/Desktop/FlipDataStore/chromedriver')

#Opening the trak.in URL
driver.get("https://trak.in/")      

# to maximize the browser window
driver.maximize_window()

In [321]:
#clicking on the Funding Deals tab
fundingdeals=driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a/i').click()


In [ ]:
#Closing the Ad
CloseAds=driver.find_element_by_xpath('//div[@class="ns-gu550-e-1 column-container full-height"]/div[2]/div/div/div[3]/div/span').click()

In [362]:
data={}   #Creating a dictionary to store as key value pairs

#Dataframe with final column names to store the result set
dealdf=pd.DataFrame(columns=['Sr.No', 'Date', 'StartupName', 'Industry/Vertical', 'Sub-Vertical', 'City/Location', 'InvestorName', 'InvestmentType', 'Amount(inUSD)'])

#July 2020 funding deals
deal1=driver.find_elements_by_xpath('//table[@id="tablepress-48"]') 
#August 2020 funding deals
deal2=driver.find_elements_by_xpath('//table[@id="tablepress-49"]')
#September 2020 funding deals
deal3=driver.find_elements_by_xpath('//table[@id="tablepress-50"]')

#Making a list to iterate by the for loop
deals=[deal1, deal2, deal3]

#deals list is passed
for deal in deals: 
    #Iterating through each of the deals
    for dl in deal:
        #Fetching the data from the deals table
        for d in dl.find_elements_by_xpath('tbody[@class="row-hover"]/tr'):
            #Serial Number
            data['Sr.No']=d.find_element_by_xpath('td[@class="column-1"]').text
            #Date
            data['Date']=d.find_element_by_xpath('td[@class="column-2"]').text
            #Startup Name
            data['StartupName']=d.find_element_by_xpath('td[@class="column-3"]').text
            #Sub-Vertical
            data['Sub-Vertical']=d.find_element_by_xpath('td[@class="column-4"]').text
            #Industry/Vertical
            data['Industry/Vertical']=d.find_element_by_xpath('td[@class="column-5"]').text
            #City/Location
            data['City/Location']=d.find_element_by_xpath('td[@class="column-6"]').text
            #Investor Name
            data['InvestorName']=d.find_element_by_xpath('td[@class="column-7"]').text
            #Investment type
            data['InvestmentType']=d.find_element_by_xpath('td[@class="column-8"]').text
            #Amount in USD
            data['Amount(inUSD)']=d.find_element_by_xpath('td[@class="column-9"]').text

        
            #Converting the dictionary data to DataFrame
            dealdf=dealdf.append(data,ignore_index=True)
#Viewing the final result sel with all the funding deals for second quarter (i.e. July 20 –September 20) from trak.in
dealdf 

,Sr.No,Date,StartupName,Industry/Vertical,Sub-Vertical,City/Location,InvestorName,InvestmentType,Amount(inUSD)
0,1,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,2,16/07/2020,Vedantu,Online Tutoring,EduTech,Bangalore,Coatue Management,Series D,"100,000,000"
2,3,16/07/2020,Crio,Learning Platform for Developers,EduTech,Bangalore,021 Capital,pre-Series A,"934,160"
3,4,14/07/2020,goDutch,Group Payments,FinTech,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,5,13/07/2020,Mystifly,"Ticketing, Airline Retailing, and Post-Ticketi...",Airfare Marketplace,Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,6,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,7,10/07/2020,gigIndia,"Crowd Sourcing, Freelance",Marketplace,Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,8,15/07/2020,PumPumPum,Used Car-leasing platform,Automotive Rental,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,9,14/07/2020,FLYX,Streaming Social Network,OTT Player,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,10,13/07/2020,Open Appliances Pvt. Ltd.,Internet-of-Things Security Solutions,Information Technology,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


###### Assignment 3.7:
    Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [525]:
#Activating the Chrome Browser
driver=webdriver.Chrome('C:/Users/ak38/Desktop/FlipDataStore/chromedriver')

#Opening the trak.in URL
driver.get("https://digit.in/")      

# to maximize the browser window
driver.maximize_window()

In [526]:
#Getting the laptop URL as the click is not working via the code
#laptops=driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()
laptops=driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').get_attribute('href')
laptops

'https://www.digit.in/laptops/'

In [527]:
driver.get(laptops)
time.sleep(3)

Glap=driver.find_element_by_xpath('//ul[@class="list-unstyled sidebar-list"]/li[10]/a').get_attribute('href')
Glap

'https://www.digit.in/top-products/best-gaming-laptops-40.html'

In [528]:
driver.get(Glap)

In [540]:
#Creating the lists to store the extracted values
pname=[]
pseller=[]
pprice=[]

#deriving the product name using the below elements
name=driver.find_elements_by_xpath('//table[@id="summtable"]/tbody/tr/td[1]')
for i in name:   ##Iterating through the Product names
    pname.append(i.text)

#deriving the Seller data using the below elements
sell=driver.find_elements_by_xpath('//td[@class="smmerchant"]')
for j in sell: #Iterating through the seller elements
    pseller.append(j.text)

#deriving the Price data using the below elements
Price=driver.find_elements_by_xpath('//table[@id="summtable"]/tbody/tr/td[3]')
len(Price)
for k in Price: #Iterating through the prices
    pprice.append(k.text)
pprice

#displaying the lists
pname, pseller, pprice

(['Alienware Area 51M R2',
  'Alienware m15 R3',
  'ASUS ROG Strix Scar 15',
  'Asus ROG Zephyrus G14',
  'Lenovo Legion 5i',
  'ASUS ROG ZEPHYRUS DUO 15',
  'Acer Aspire 7 gaming'],
 ['N/A', 'N/A', 'N/A', 'amazon', 'N/A', 'amazon', 'N/A'],
 ['N/A', '₹341990', 'N/A', '₹164990', '₹71990', '₹199990', '₹56990'])

In [545]:
#Lets make it dataframe for better visualization
import pandas as pd
data=pd.DataFrame({})
data['Prodduct_Name']=pname
data['Seller']=pseller
data['Price']=pprice
data

,Prodduct_Name,Seller,Price
0,Alienware Area 51M R2,N/A,N/A
1,Alienware m15 R3,N/A,₹341990
2,ASUS ROG Strix Scar 15,N/A,N/A
3,Asus ROG Zephyrus G14,amazon,₹164990
4,Lenovo Legion 5i,N/A,₹71990
5,ASUS ROG ZEPHYRUS DUO 15,amazon,₹199990
6,Acer Aspire 7 gaming,N/A,₹56990
